In [68]:
# Step 1: Load and preprocess the dataset
# Assuming you have a dataset with image paths and corresponding labels
# Load the dataset and preprocess the images (resize, normalize, etc.)
# Split the dataset into training and testing sets

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = "./dataset1/train"
test_path = "./dataset1/test"


train_gen = ImageDataGenerator(     
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,                
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')
test_gen = ImageDataGenerator(rescale=(1./255))
     
train = train_gen.flow_from_directory(train_path,
                                      target_size=(48, 48),
                                      class_mode='categorical', 
                                      color_mode = "grayscale",
                                      batch_size=64)
test = test_gen.flow_from_directory(test_path,
                                    target_size=(48, 48),
                                    class_mode='categorical',
                                    color_mode="grayscale",
                                      batch_size=64)
# train_gen = ImageDataGenerator(rescale=(1./225),
#                               rotation_range=40,
#                               width_shift_range=0.2,                
#                               height_shift_range=0.2,
#                               shear_range=0.2,
#                               zoom_range=0.2,
#                               fill_mode='nearest'))
# test_gen = ImageDataGenerator(rescale=(1./255))
     

# train = train_gen.flow_from_directory(train_path,
#                                       target_size=(48, 48),
#                                       color_mode="grayscale",
#                                       class_mode='categorical', 
#                                       batch_size=64)
# test = test_gen.flow_from_directory(test_path,
#                                     target_size=(48, 48),
#                                       class_mode='categorical', 
#                                     color_mode="grayscale",
#                                       batch_size=64)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [69]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential

# Step 2: Define the CNN architecture
emodel = Sequential()
emodel.add(Convolution2D(24,(3,3),activation='relu',input_shape=(48, 48, 1)))
emodel.add(MaxPooling2D(pool_size=(2,2)))
emodel.add(Convolution2D(32,(3,3),activation='relu'))
emodel.add(MaxPooling2D(pool_size=(2,2)))
emodel.add(Convolution2D(32,(3,3),activation='relu'))
emodel.add(MaxPooling2D(pool_size=(2,2)))
emodel.add(Convolution2D(48,(3,3),activation='relu'))
emodel.add(MaxPooling2D(pool_size=(2,2)))

emodel.add(Flatten())
emodel.add(Dense(62,activation='relu'))
emodel.add(Dense(32,activation='relu'))
emodel.add(Dense(16,activation='relu'))
emodel.add(Dense(7,activation='softmax'))

# Step 3: Compile the model
emodel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Step 4: Train the model
emodel.fit(train,batch_size=64,validation_data=test,epochs=100)

Epoch 1/100
449/449 [==============================] - 37s 80ms/step - loss: 1.8280 - accuracy: 0.2491 - val_loss: 1.8151 - val_accuracy: 0.2471
Epoch 2/100
449/449 [==============================] - 36s 80ms/step - loss: 1.8057 - accuracy: 0.2518 - val_loss: 1.7904 - val_accuracy: 0.2558
Epoch 3/100
449/449 [==============================] - 36s 80ms/step - loss: 1.7934 - accuracy: 0.2528 - val_loss: 1.7845 - val_accuracy: 0.2471
Epoch 4/100
449/449 [==============================] - 36s 80ms/step - loss: 1.7862 - accuracy: 0.2551 - val_loss: 1.7762 - val_accuracy: 0.2517
Epoch 5/100
449/449 [==============================] - 36s 80ms/step - loss: 1.7754 - accuracy: 0.2587 - val_loss: 1.7255 - val_accuracy: 0.2920
Epoch 6/100
449/449 [==============================] - 36s 80ms/step - loss: 1.7465 - accuracy: 0.2763 - val_loss: 1.6955 - val_accuracy: 0.3073
Epoch 7/100
449/449 [==============================] - 36s 81ms/step - loss: 1.7097 - accuracy: 0.2976 - val_loss: 1.6367 - val_ac

449/449 [==============================] - 37s 83ms/step - loss: 1.3065 - accuracy: 0.4977 - val_loss: 1.1796 - val_accuracy: 0.5468
Epoch 58/100
449/449 [==============================] - 37s 81ms/step - loss: 1.3113 - accuracy: 0.4943 - val_loss: 1.1906 - val_accuracy: 0.5435
Epoch 59/100
449/449 [==============================] - 37s 81ms/step - loss: 1.3006 - accuracy: 0.4981 - val_loss: 1.2122 - val_accuracy: 0.5375
Epoch 60/100
449/449 [==============================] - 37s 81ms/step - loss: 1.2996 - accuracy: 0.5027 - val_loss: 1.2702 - val_accuracy: 0.5160
Epoch 61/100
449/449 [==============================] - 37s 82ms/step - loss: 1.3039 - accuracy: 0.5004 - val_loss: 1.2146 - val_accuracy: 0.5316
Epoch 62/100
449/449 [==============================] - 37s 82ms/step - loss: 1.2963 - accuracy: 0.5012 - val_loss: 1.1817 - val_accuracy: 0.5486
Epoch 63/100
449/449 [==============================] - 37s 82ms/step - loss: 1.2965 - accuracy: 0.5015 - val_loss: 1.1810 - val_accuracy

In [38]:
# DETECTION CODE

# model.save('model.h5')
from keras.models import load_model

import cv2
import mediapipe as mp
import os
import numpy as np
from tensorflow.keras.preprocessing import image

# Initialize MediaPipe hand detection
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

# Loading the pre-trained MediaPipe hand detection model
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1, min_detection_confidence=0.3)

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture frame from the camera
    ret, frame = cap.read()

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand detection
    results = hands.process(frame_rgb)

    # If hand is detected, crop and save the image
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Get bounding box coordinates of the hand
        x_min, y_min, x_max, y_max = detect_hand(hand_landmarks, frame)

        # If the bounding box is valid, crop and save the image
        if x_min != -1 and y_min != -1:
            cropped_image = frame[y_min:y_max, x_min:x_max]
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing_styles.get_default_hand_landmarks_style(),
                                         mp_drawing_styles.get_default_hand_connections_style())
            img1 = cv2.resize(cropped_image,(224,224),interpolation=cv2.INTER_AREA)
            img1 = image.img_to_array(img1)
            img1 = np.expand_dims(img1,axis=0)
            pred = np.argmax(emodel.predict(img1))

            output = ['A','B','C','D','E','F','G','H','I','K','L','O','P','Q','R','S','U','V','W','X','Y']

            cv2.putText(frame,str(output[pred]),(x_min,y_min-10),cv2.FONT_HERSHEY_DUPLEX, 1.3, (0, 0, 0), 3,cv2.LINE_AA)
#     # Draw hand landmarks and bounding box on the frame
#     if results.multi_hand_landmarks:
#         for hand_landmarks in results.multi_hand_landmarks:
#             mp_drawing.draw_landmarks(
#                 frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        if x_min != -1 and y_min != -1:
            cv2.rectangle(frame, (x_min, y_min),
                          (x_max, y_max), (0, 255, 0), 2)
    
    
    cv2.imshow('Real-time Hand Detection', frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [62]:
import cv2
import mediapipe as mp
import os

# Initialize MediaPipe FaceMesh
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Create the directory to save the images
save_dir = "./dataset1/train"
os.makedirs(save_dir, exist_ok=True)

# Load the video or use webcam
video_capture = cv2.VideoCapture(0)  # Replace 0 with the video file path if using a video

# Initialize FaceMesh
with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:

    count = 0  # Counter for the number of captured images

    while count < 100:
        # Read each frame of the video
        ret, frame = video_capture.read()

        # Break loop if video has ended
        if not ret:
            break

        # Convert the image color space from BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with FaceMesh
        results = face_mesh.process(rgb_frame)

        # Draw face landmarks and bounding box on the frame
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                )

                # Extract face region coordinates
                image_height, image_width, _ = frame.shape
                face_landmark_coords = []
                for landmark in face_landmarks.landmark:
                    x = int(landmark.x * image_width)
                    y = int(landmark.y * image_height)
                    face_landmark_coords.append((x, y))

                # Calculate bounding box for face region
                x_min = min(face_landmark_coords, key=lambda x: x[0])[0]
                y_min = min(face_landmark_coords, key=lambda x: x[1])[1]
                x_max = max(face_landmark_coords, key=lambda x: x[0])[0]
                y_max = max(face_landmark_coords, key=lambda x: x[1])[1]

                # Calculate center of the bounding box
                center_x = (x_min + x_max) // 2
                center_y = (y_min + y_max) // 2

                # Calculate the coordinates for cropping the face region
                crop_size = 112  # Half of the desired crop size (224/2)
                crop_x_min = max(0, center_x - crop_size)
                crop_y_min = max(0, center_y - crop_size)
                crop_x_max = min(image_width, center_x + crop_size)
                crop_y_max = min(image_height, center_y + crop_size)

                # Crop the face region
                face_crop = frame[crop_y_min:crop_y_max, crop_x_min:crop_x_max]

                # Resize the cropped face region to the desired size (224x224)
                face_crop_resized = cv2.resize(face_crop, (224, 224))

                # Save the cropped face region as an image
                image_name = f"image{count+1}.jpg"
                save_path = os.path.join(save_dir, image_name)
                cv2.imwrite(save_path, face_crop_resized)

                count += 1

        # Display the resulting frame with face landmarks
        cv2.imshow('Face Landmarks Detection', frame)

        # Break loop if 'q' is pressed
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the windows
    video_capture.release()
    cv2.destroyAllWindows()
